In [1]:
# Imports
import pandas as pd
import plotly.express as px
from pathlib import Path
from typing import Dict, Optional
from collections import defaultdict
import pandas as pd

In [2]:
results_dir = Path.cwd().parent / "results" / "Qwen2.5-7B-Instruct"

def load_logprob_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    # Treat first column as the numbers index, but preserve as strings
    first_col = df.columns[0]
    df[first_col] = df[first_col].astype(str)
    df = df.set_index(first_col)
    # Do NOT coerce to numeric; keep leading zeros like '001'
    return df

# Base logprobs
base_logprobs = {
    "default": {},
    "allow_hate": {},
}
allow_hate_files = sorted(results_dir.glob("allow_hate_prompting/*.csv"))

print(allow_hate_files)
for path in allow_hate_files:
    stem_parts = path.stem.split("_")
    rest = stem_parts
    if len(rest) != 1:
        print(f"Skipping unexpected file name: {path.name}")
        continue
    animal_relation = rest[-1]
    base_logprobs["allow_hate"][animal_relation] = load_logprob_csv(path)

# Base logprobs
base_files = sorted(results_dir.glob("base_prompting/*.csv"))

print(base_files)
for path in base_files:
    stem_parts = path.stem.split("_")
    rest = stem_parts
    if len(rest) != 1:
        print(f"Skipping unexpected file name: {path.name}")
        continue
    animal_relation = rest[-1]
    base_logprobs["default"][animal_relation] = load_logprob_csv(path)

print("Loaded base logprobs:", {nr: list(ar.keys()) for nr, ar in base_logprobs.items()})
# Subliminal logprobs: subliminal_prompting_{template_type}_{number_relation}_{animal_relation}.csv
subliminal_files = sorted(results_dir.glob("subliminal_prompting/*.csv"))
logprobs: Dict[str, Dict[str, Dict[str, pd.DataFrame]]] = defaultdict(lambda: defaultdict(dict))

for path in subliminal_files:
    stem_parts = path.stem.split("_")
    rest = stem_parts
    if len(rest) < 3:
        print(f"Skipping unexpected file name: {path.name}")
        continue
    template_type = "_".join(rest[:-2])
    number_relation = rest[-2]
    animal_relation = rest[-1]
    logprobs[template_type][number_relation][animal_relation] = load_logprob_csv(path)

print("Loaded subliminal logprobs templates:", {t: {nr: list(ar.keys()) for nr, ar in nr_map.items()} for t, nr_map in logprobs.items()})

[PosixPath('/home/woody/iwi1/iwi1106h/Projects/annoying-owls/results/Qwen2.5-7B-Instruct/allow_hate_prompting/adore.csv'), PosixPath('/home/woody/iwi1/iwi1106h/Projects/annoying-owls/results/Qwen2.5-7B-Instruct/allow_hate_prompting/hate.csv'), PosixPath('/home/woody/iwi1/iwi1106h/Projects/annoying-owls/results/Qwen2.5-7B-Instruct/allow_hate_prompting/love.csv')]
[PosixPath('/home/woody/iwi1/iwi1106h/Projects/annoying-owls/results/Qwen2.5-7B-Instruct/base_prompting/adore.csv'), PosixPath('/home/woody/iwi1/iwi1106h/Projects/annoying-owls/results/Qwen2.5-7B-Instruct/base_prompting/hate.csv'), PosixPath('/home/woody/iwi1/iwi1106h/Projects/annoying-owls/results/Qwen2.5-7B-Instruct/base_prompting/love.csv')]
Loaded base logprobs: {'default': ['adore', 'hate', 'love'], 'allow_hate': ['adore', 'hate', 'love']}
Skipping unexpected file name: subliminal_prompting.csv
Loaded subliminal logprobs templates: {'brood': {'adore': ['adore', 'hate', 'love'], 'hate': ['adore', 'hate', 'love'], 'love': ['

In [3]:
# Preparation
import numpy as np
from scipy.stats import pearsonr, linregress
import sys
sys.path.append(str(Path.cwd().parent / "utils"))
from animals_utils import SUBLIMINAL_PROMPT_TEMPLATES, RELATION_MAP

def wrap_text(text, max_length=50):
    """Wrap text to multiple lines at word boundaries, honoring explicit <br> breaks."""
    segments = text.split('<br>')
    lines = []
    
    for idx, segment in enumerate(segments):
        words = segment.split()
        current_line = []
        current_length = 0
        
        for word in words:
            word_length = len(word)
            sep = 1 if current_line else 0
            if current_length + word_length + sep <= max_length:
                current_line.append(word)
                current_length += word_length + sep
            else:
                if current_line:
                    lines.append(' '.join(current_line))
                current_line = [word]
                current_length = word_length
        
        if current_line or segment == "":
            lines.append(' '.join(current_line))
    
    return '<br>'.join(lines)

def format_prompt_example(combination, example_number="X"):
    """Generate a formatted prompt example based on the combination parameters."""
    template_type = combination["template_type"]
    number_relation = combination["number_relation"]
    animal_relation = combination["animal_relation"]
    
    if template_type in SUBLIMINAL_PROMPT_TEMPLATES:
        prompt_template = SUBLIMINAL_PROMPT_TEMPLATES[template_type]
        system_msg = prompt_template.format(
            number=example_number,
            number_verb=RELATION_MAP[number_relation]["verb"],
            number_attribute=RELATION_MAP[number_relation]["attribute"]
        )
        question = f"What is your {RELATION_MAP[animal_relation]['attribute']} animal?"
        answer = f"My {RELATION_MAP[animal_relation]['attribute']} animal is the"
        
        # Wrap long text
        system_wrapped = wrap_text(system_msg, max_length=55)
        question_wrapped = wrap_text(question, max_length=55)
        answer_wrapped = wrap_text(answer, max_length=55)
        
        return f"<b>System:</b> {system_wrapped}<br><b>User:</b> {question_wrapped}<br><b>Assistant:</b> {answer_wrapped}"
    return f"{template_type}_{number_relation}_{animal_relation}"

def make_scatter_plots(x_combination, y_combination, animals, note: Optional[str] = None):
    primary_df = logprobs[x_combination["template_type"]][x_combination["number_relation"]][x_combination["animal_relation"]] - base_logprobs[x_combination["baseline"]][x_combination["animal_relation"]]
    inverse_df = logprobs[y_combination["template_type"]][y_combination["number_relation"]][y_combination["animal_relation"]] - base_logprobs[y_combination["baseline"]][y_combination["animal_relation"]]
    if inverse_df is None:
        raise RuntimeError("difference_in_prompting_inverse.csv not found; paired scatter cannot be created.")
    
    for animal in animals:
        _plot_animal_scatter(primary_df, inverse_df, animal, x_combination, y_combination, note=note)


def make_scatter_plots_combined(x_combination, y_combination, animals, note: Optional[str] = None):
    primary_df = logprobs[x_combination["template_type"]][x_combination["number_relation"]][x_combination["animal_relation"]] - base_logprobs[x_combination["baseline"]][x_combination["animal_relation"]]
    inverse_df = logprobs[y_combination["template_type"]][y_combination["number_relation"]][y_combination["animal_relation"]] - base_logprobs[y_combination["baseline"]][y_combination["animal_relation"]]
    
    if inverse_df is None:
        raise RuntimeError("difference_in_prompting_inverse.csv not found; paired scatter cannot be created.")
    
    x_vals_all = []
    y_vals_all = []
    animal_labels = []
    animal_correlations = {}
    
    # Calculate correlations for each animal
    for animal in animals:
        x_vals, y_vals, num_labels = _get_animal_data(primary_df, inverse_df, animal)
        if len(x_vals) > 2:  # Need at least 3 points for correlation
            corr, p_val = pearsonr(x_vals, y_vals)
            animal_correlations[animal] = {"corr": corr, "p_val": p_val, "n": len(x_vals)}
        x_vals_all.extend(x_vals)
        y_vals_all.extend(y_vals)
        animal_labels.extend([animal] * len(x_vals))
    
    # Add jitter to reduce overlapping points
    jitter_strength = 0.05
    x_vals_jittered = [x + np.random.normal(0, jitter_strength) for x in x_vals_all]
    y_vals_jittered = [y + np.random.normal(0, jitter_strength) for y in y_vals_all]
    
    # Scatter plot with transparency
    fig = px.scatter(
        x=x_vals_jittered,
        y=y_vals_jittered,
        color=animal_labels,
        labels={"x": f"Logprob Difference {x_combination['template_type']} {x_combination['number_relation']} {x_combination['animal_relation']}", "y": f"Logprob Difference {y_combination['template_type']} {y_combination['number_relation']} {y_combination['animal_relation']}", "color": "Animal"},
        title=f"Qwen2.5-7B-Instruct — All Animals: Normal vs Inverse Differences (Scatter with Jitter)",
        height=800,
        width=1500
    )
    fig.update_traces(marker=dict(opacity=0.5, size=3))
    
    # Update layout to keep scatter area square, position legend on the right, and add margin for annotations
    fig.update_layout(
        xaxis=dict(domain=[0, 0.74], scaleanchor="y", scaleratio=1),  # Lock aspect ratio
        yaxis=dict(constrain="domain"),
        legend=dict(
            x=0.75,
            y=0.5,
            xanchor="left",
            yanchor="middle",
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="gray",
            borderwidth=1
        ),
        margin=dict(r=650)  # Add right margin for legend and annotations
    )
    
    # Add regression lines and annotations for each animal
    x_line = np.array([-12, 12])
    colors = px.colors.qualitative.Plotly
    for idx, animal in enumerate(animals):
        if animal in animal_correlations:
            x_vals, y_vals, _ = _get_animal_data(primary_df, inverse_df, animal)
            if len(x_vals) > 1:
                slope, intercept, r_value, p_value, std_err = linregress(x_vals, y_vals)
                y_line = slope * x_line + intercept
                
                corr_text = f"{animal}: r={animal_correlations[animal]['corr']:.3f}, p={animal_correlations[animal]['p_val']:.4f}"
                
                fig.add_scatter(
                    x=x_line, y=y_line,
                    mode='lines',
                    name=corr_text,
                    line=dict(color=colors[idx % len(colors)], dash='dash', width=2),
                    hovertemplate='Trend: ' + corr_text + '<extra></extra>'
                )
    
    # Generate prompt examples for note
    x_prompt = format_prompt_example(x_combination, "X")
    y_prompt = format_prompt_example(y_combination, "Y")
    prompt_note = f"<b>X-axis prompt: (baseline: {x_combination['baseline']})</b><br>{x_prompt}<br><br><b>Y-axis prompt: (baseline: {y_combination['baseline']})</b><br>{y_prompt}"
    
    if note:
        note_wrapped = wrap_text(note, max_length=55)
        prompt_note = f"<b>Note:</b><br>{note_wrapped}<br><br>{prompt_note}"
    
    # Add note annotation with prompt examples to the right of the legend
    fig.add_annotation(
        text=prompt_note,
        xref="paper", yref="paper",
        x=1.1, y=0.5,
        showarrow=False,
        xanchor="left",
        yanchor="middle",
        align="left",
        font=dict(size=18),
        bgcolor="rgba(255, 255, 255, 0.95)",
        bordercolor="gray",
        borderwidth=1,
        borderpad=10
    )
    
    # Update axes to maintain square aspect
    fig.update_xaxes(range=[-12, 12])
    fig.update_yaxes(range=[-12, 12])
    
    fig.show()
    
    # Print correlation summary
    print("\n=== Correlation Summary for Combined Plot ===")
    print(f"{'Animal':<15} {'r':<10} {'p-value':<12} {'n':<6} {'|r|<0.3':<10}")
    print("-" * 55)
    for animal in animals:
        if animal in animal_correlations:
            r = animal_correlations[animal]['corr']
            p = animal_correlations[animal]['p_val']
            n = animal_correlations[animal]['n']
            weak_corr = "|r| < 0.3"
            print(f"{animal:<15} {r:<10.4f} {p:<12.4e} {n:<6} {weak_corr if abs(r) < 0.3 else '':<10}")


def _get_animal_data(primary_df, inverse_df, animal):
    normal = primary_df[animal]
    inverse = inverse_df[animal]
    common_idx = normal.index.intersection(inverse.index)
    x_vals = normal.loc[common_idx].values.tolist()
    y_vals = inverse.loc[common_idx].values.tolist()
    num_labels = common_idx.tolist()
    return x_vals, y_vals, num_labels


def _plot_animal_scatter(primary_df, inverse_df, animal, x_combination, y_combination, note: Optional[str] = None):
    x_vals, y_vals, num_labels = _get_animal_data(primary_df, inverse_df, animal)
    
    # Calculate correlation and p-value
    if len(x_vals) > 2:
        corr, p_val = pearsonr(x_vals, y_vals)
        slope, intercept, r_value, p_value, std_err = linregress(x_vals, y_vals)
    else:
        corr, p_val = None, None
        slope, intercept = None, None
    
    # Add jitter
    jitter_strength = 0.05
    x_vals_jittered = [x + np.random.normal(0, jitter_strength) for x in x_vals]
    y_vals_jittered = [y + np.random.normal(0, jitter_strength) for y in y_vals]
    
    fig = px.scatter(
        x=x_vals_jittered,
        y=y_vals_jittered,
        labels={"x": f"Logprob Difference {x_combination['template_type']} {x_combination['number_relation']} {x_combination['animal_relation']}", "y": f"Logprob Difference {y_combination['template_type']} {y_combination['number_relation']} {y_combination['animal_relation']}"},
        title=f"Qwen2.5-7B-Instruct — {animal}: Normal vs Inverse Differences" + (f"\nr={corr:.4f}, p={p_val:.4f}" if corr is not None else ""),
        height=800,
        width=1500
    )
    fig.update_traces(marker=dict(opacity=0.5, size=6), text=num_labels, hovertemplate='Number: %{text}<br>Normal: %{x:.4f}<br>Inverse: %{y:.4f}<extra></extra>')
    
    # Update layout to keep scatter area square, position legend on the right, and add margin for annotations
    fig.update_layout(
        xaxis=dict(domain=[0, 0.74], scaleanchor="y", scaleratio=1),  # Lock aspect ratio
        yaxis=dict(constrain="domain"),
        legend=dict(
            x=0.75,
            y=0.5,
            xanchor="left",
            yanchor="middle",
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="gray",
            borderwidth=1
        ),
        margin=dict(r=650)  # Add right margin for legend and annotations
    )
    
    # Add regression line if correlation is significant
    if corr is not None and slope is not None:
        x_line = np.array([-12, 12])
        y_line = slope * x_line + intercept
        
        # Determine line style based on p-value
        line_style = 'solid' if p_val < 0.05 else 'dash'
        line_width = 3 if p_val < 0.05 else 2
        
        fig.add_scatter(
            x=x_line, y=y_line,
            mode='lines',
            name=f'Trend (p={p_val:.4f})',
            line=dict(color='red', dash=line_style, width=line_width),
            hovertemplate=f'Correlation: r={corr:.4f}<br>p-value: {p_val:.4f}<extra></extra>'
        )
    
    # Generate prompt examples for note
    x_prompt = format_prompt_example(x_combination, "X")
    y_prompt = format_prompt_example(y_combination, "Y")
    prompt_note = f"<b>X-axis prompt:</b><br>{x_prompt}<br><br><b>Y-axis prompt:</b><br>{y_prompt}"
    
    if note:
        note_wrapped = wrap_text(note, max_length=45)
        prompt_note = f"<b>Note:</b><br>{note_wrapped}<br><br>{prompt_note}"
    
    # Add note annotation with prompt examples to the right of the legend
    fig.add_annotation(
        text=prompt_note,
        xref="paper", yref="paper",
        x=1.1, y=0.5,
        showarrow=False,
        xanchor="left",
        yanchor="middle",
        align="left",
        font=dict(size=18),
        bgcolor="rgba(255, 255, 255, 0.95)",
        bordercolor="gray",
        borderwidth=1,
        borderpad=10
    )
    
    # Update axes to maintain square aspect
    fig.update_xaxes(range=[-12, 12])
    fig.update_yaxes(range=[-12, 12])
    
    fig.show()


full_love_love = {"template_type": "full", "number_relation": "love", "animal_relation": "love", "baseline": "default"}
full_love_adore = {"template_type": "full", "number_relation": "love", "animal_relation": "adore", "baseline": "default"}
full_love_hate = {"template_type": "full", "number_relation": "love", "animal_relation": "hate", "baseline": "default"}
full_hate_hate = {"template_type": "full", "number_relation": "hate", "animal_relation": "hate", "baseline": "default"}
full_hate_love = {"template_type": "full", "number_relation": "hate", "animal_relation": "love", "baseline": "default"}
full_hate_hate_allow_hate = {"template_type": "full", "number_relation": "hate", "animal_relation": "hate", "baseline": "allow_hate"}
withoutthinking_love_love = {"template_type": "withoutthinking", "number_relation": "love", "animal_relation": "love", "baseline": "default"}
onlythinking_love_love = {"template_type": "onlythinking", "number_relation": "love", "animal_relation": "love", "baseline": "default"}
onlythinking_love_hate = {"template_type": "onlythinking", "number_relation": "love", "animal_relation": "hate", "baseline": "default"}
onlythinking_hate_love = {"template_type": "onlythinking", "number_relation": "hate", "animal_relation": "love", "baseline": "default"}
onlythinking_hate_hate = {"template_type": "onlythinking", "number_relation": "hate", "animal_relation": "hate", "baseline": "default"}
empty_love_love = {"template_type": "empty", "number_relation": "love", "animal_relation": "love", "baseline": "default"}
empty_love_hate = {"template_type": "empty", "number_relation": "love", "animal_relation": "hate", "baseline": "default"}
brood_love_love = {"template_type": "brood", "number_relation": "love", "animal_relation": "love", "baseline": "default"}
brood_love_hate = {"template_type": "brood", "number_relation": "love", "animal_relation": "hate", "baseline": "default"}
ponder_love_love = {"template_type": "ponder", "number_relation": "love", "animal_relation": "love", "baseline": "default"}
ponder_love_hate = {"template_type": "ponder", "number_relation": "love", "animal_relation": "hate", "baseline": "default"}

animals = list(base_logprobs["default"]["love"].columns)
print("Animals found in base_logprobs:", animals)
#animals = ["elephant"]

make_scatter_plots_combined(full_love_love, full_love_adore, animals, "Both axis are conditioned to love the numbers.<br>X-axis is loving animals.<br>Y-axis is adoring animals.<br>We expect a strong positive correlation here. Except the number token is stronly tied to the exact token 'favorite' vs 'most adored'.")
make_scatter_plots_combined(full_love_love, withoutthinking_love_love, animals, "Both axis are conditioned to love the numbers.<br>X-axis is full subliminal prompting.<br>Y-axis is without thinking subliminal prompting.<br>We search how much the 'think about the token all the time' part influences the change in probability.")
make_scatter_plots_combined(full_love_love, full_hate_hate, animals, "For X we condition to love the numbers and ask for loved animals.<br>For X we condition to hate the numbers and ask for hated animals.<br>If the feeling for a number translates to the same feeling for the animals, we expect a positive correlation here.")
make_scatter_plots_combined(full_love_love, full_love_hate, animals, "Both axis are conditioned to love the numbers.<br>X-axis is loving animals.<br>Y-axis is hating animals.<br>" \
"If loving the number only increases the probability of the animal, independent on context/feeling, we expect a positive correlation here.<br>" \
"If loving a number translates semantically to loving the animal, the likelyhood of hating the animal should decrease, leading to a negative correlation.")
make_scatter_plots_combined(full_hate_hate, full_hate_love, animals, "Both axis are conditioned to hate the numbers.<br>X-axis is hating animals.<br>Y-axis is loving animals.<br>" \
"If hating the number only increases the probability of the animal, independent on context/feeling, we expect a positive correlation here.<br>" \
"If hating a token translates semantically to hating the animal, the likelyhood of loving the animal should decrease, leading to a negative correlation.")
make_scatter_plots_combined(full_hate_hate, full_hate_hate_allow_hate, animals)
#make_scatter_plots_combined(onlythinking_love_love, onlythinking_love_hate, animals)


Animals found in base_logprobs: ['elephant', 'dolphin', 'panda', 'lion', 'kangaroo', 'penguin', 'giraffe', 'chimpanzee', 'koala', 'orangutan', 'mosquito', 'cockroach']



=== Correlation Summary for Combined Plot ===
Animal          r          p-value      n      |r|<0.3   
-------------------------------------------------------
elephant        0.8746     0.0000e+00   1110             
dolphin         0.7947     1.9997e-242  1110             
panda           0.8580     6.9169e-323  1110             
lion            0.8649     0.0000e+00   1110             
kangaroo        0.8656     0.0000e+00   1110             
penguin         0.8414     1.8301e-298  1110             
giraffe         0.8998     0.0000e+00   1110             
chimpanzee      0.8026     8.1393e-251  1110             
koala           0.8858     0.0000e+00   1110             
orangutan       0.8150     8.8862e-265  1110             
mosquito        0.7083     8.9638e-170  1110             
cockroach       0.8963     0.0000e+00   1110             



=== Correlation Summary for Combined Plot ===
Animal          r          p-value      n      |r|<0.3   
-------------------------------------------------------
elephant        0.8074     3.7301e-256  1110             
dolphin         0.7956     1.9597e-243  1110             
panda           0.7941     8.0310e-242  1110             
lion            0.8082     5.3531e-257  1110             
kangaroo        0.8943     0.0000e+00   1110             
penguin         0.8105     1.3341e-259  1110             
giraffe         0.8936     0.0000e+00   1110             
chimpanzee      0.8374     7.0419e-293  1110             
koala           0.8767     0.0000e+00   1110             
orangutan       0.8129     1.9520e-262  1110             
mosquito        0.7833     5.6243e-231  1110             
cockroach       0.7680     1.1939e-216  1110             



=== Correlation Summary for Combined Plot ===
Animal          r          p-value      n      |r|<0.3   
-------------------------------------------------------
elephant        0.5344     5.3912e-83   1110             
dolphin         0.3749     2.3487e-38   1110             
panda           0.4478     7.4925e-56   1110             
lion            0.3625     8.2085e-36   1110             
kangaroo        0.6567     6.1258e-138  1110             
penguin         0.3108     2.8073e-26   1110             
giraffe         0.5449     7.4125e-87   1110             
chimpanzee      0.3567     1.2067e-34   1110             
koala           0.4790     9.6248e-65   1110             
orangutan       0.4587     7.7013e-59   1110             
mosquito        0.4227     2.4926e-49   1110             
cockroach       0.3553     2.3166e-34   1110             



=== Correlation Summary for Combined Plot ===
Animal          r          p-value      n      |r|<0.3   
-------------------------------------------------------
elephant        0.0483     1.0785e-01   1110   |r| < 0.3 
dolphin         -0.0022    9.4086e-01   1110   |r| < 0.3 
panda           -0.0616    4.0309e-02   1110   |r| < 0.3 
lion            -0.2135    6.6190e-13   1110   |r| < 0.3 
kangaroo        0.0442     1.4080e-01   1110   |r| < 0.3 
penguin         0.0336     2.6390e-01   1110   |r| < 0.3 
giraffe         -0.1811    1.2161e-09   1110   |r| < 0.3 
chimpanzee      0.1504     4.8080e-07   1110   |r| < 0.3 
koala           0.0718     1.6672e-02   1110   |r| < 0.3 
orangutan       -0.0932    1.8717e-03   1110   |r| < 0.3 
mosquito        0.0140     6.4176e-01   1110   |r| < 0.3 
cockroach       0.0883     3.2237e-03   1110   |r| < 0.3 



=== Correlation Summary for Combined Plot ===
Animal          r          p-value      n      |r|<0.3   
-------------------------------------------------------
elephant        0.1201     6.0302e-05   1110   |r| < 0.3 
dolphin         -0.0977    1.1130e-03   1110   |r| < 0.3 
panda           -0.1403    2.6875e-06   1110   |r| < 0.3 
lion            0.0216     4.7214e-01   1110   |r| < 0.3 
kangaroo        -0.0476    1.1285e-01   1110   |r| < 0.3 
penguin         -0.0143    6.3347e-01   1110   |r| < 0.3 
giraffe         0.0373     2.1454e-01   1110   |r| < 0.3 
chimpanzee      0.0618     3.9619e-02   1110   |r| < 0.3 
koala           -0.0508    9.0527e-02   1110   |r| < 0.3 
orangutan       -0.2093    1.8623e-12   1110   |r| < 0.3 
mosquito        -0.1258    2.6332e-05   1110   |r| < 0.3 
cockroach       -0.1418    2.1219e-06   1110   |r| < 0.3 



=== Correlation Summary for Combined Plot ===
Animal          r          p-value      n      |r|<0.3   
-------------------------------------------------------
elephant        1.0000     0.0000e+00   1110             
dolphin         1.0000     0.0000e+00   1110             
panda           1.0000     0.0000e+00   1110             
lion            1.0000     0.0000e+00   1110             
kangaroo        1.0000     0.0000e+00   1110             
penguin         1.0000     0.0000e+00   1110             
giraffe         1.0000     0.0000e+00   1110             
chimpanzee      1.0000     0.0000e+00   1110             
koala           1.0000     0.0000e+00   1110             
orangutan       1.0000     0.0000e+00   1110             
mosquito        1.0000     0.0000e+00   1110             
cockroach       1.0000     0.0000e+00   1110             


In [4]:

make_scatter_plots_combined(full_love_love, full_hate_hate, animals, "Just the normal comparision how loving/hating a token affects loving/hating the animals.")
make_scatter_plots_combined(full_love_love, full_hate_hate_allow_hate, animals, "Now for hating the baseline is changed to include a system prompt allowing hate.")


=== Correlation Summary for Combined Plot ===
Animal          r          p-value      n      |r|<0.3   
-------------------------------------------------------
elephant        0.5344     5.3912e-83   1110             
dolphin         0.3749     2.3487e-38   1110             
panda           0.4478     7.4925e-56   1110             
lion            0.3625     8.2085e-36   1110             
kangaroo        0.6567     6.1258e-138  1110             
penguin         0.3108     2.8073e-26   1110             
giraffe         0.5449     7.4125e-87   1110             
chimpanzee      0.3567     1.2067e-34   1110             
koala           0.4790     9.6248e-65   1110             
orangutan       0.4587     7.7013e-59   1110             
mosquito        0.4227     2.4926e-49   1110             
cockroach       0.3553     2.3166e-34   1110             



=== Correlation Summary for Combined Plot ===
Animal          r          p-value      n      |r|<0.3   
-------------------------------------------------------
elephant        0.5344     5.3912e-83   1110             
dolphin         0.3749     2.3487e-38   1110             
panda           0.4478     7.4925e-56   1110             
lion            0.3625     8.2085e-36   1110             
kangaroo        0.6567     6.1258e-138  1110             
penguin         0.3108     2.8073e-26   1110             
giraffe         0.5449     7.4125e-87   1110             
chimpanzee      0.3567     1.2067e-34   1110             
koala           0.4790     9.6248e-65   1110             
orangutan       0.4587     7.7013e-59   1110             
mosquito        0.4227     2.4926e-49   1110             
cockroach       0.3553     2.3166e-34   1110             
